In [1]:
import pandas as pd
import numpy as np
import gensim
import re

In [2]:
word_vector = './wordvector/tencent/tencent-ailab-embedding-zh-d100-v0.2.0-s.txt'
model = gensim.models.KeyedVectors.load_word2vec_format(word_vector,binary=False)


In [9]:
model.get_vector('qq')

array([-0.093392,  0.220605, -0.124798,  0.350253, -0.16143 ,  0.308159,
       -0.211122, -0.32076 ,  0.361758,  0.040447, -0.471041, -0.215215,
       -0.070266, -0.326788, -0.685831,  0.300579,  0.35874 , -0.384775,
       -0.202867,  0.373543,  0.006979, -0.502948,  0.088676,  0.026263,
        0.194542, -0.091298, -0.016306,  0.77258 , -0.266076,  0.105592,
        0.66647 ,  0.104468,  0.271039, -0.06656 ,  0.101982,  0.292377,
        0.208856,  0.196187,  0.277172, -0.171319,  0.07981 ,  0.330285,
       -0.194032, -0.0612  , -0.363808,  0.388273, -0.05781 ,  0.234992,
        0.132965,  0.20248 ,  0.095576, -0.285402, -0.236796, -0.173528,
       -0.346682, -0.381233,  0.157966, -0.160209, -0.056127, -0.39622 ,
        0.042586,  0.307616, -0.045831, -0.213859, -0.027358, -0.579476,
        0.42324 ,  0.020658,  0.020944, -0.510335, -0.171755,  0.276625,
       -0.012641, -0.238969, -0.387946, -0.055795,  0.46416 , -0.403004,
        0.026485,  0.390238,  0.281626,  0.673694, 

In [7]:
data = pd.read_csv('train_tokenized.csv')
f = open('speakers.txt','r')
name_list = f.readlines()
for i in range(len(name_list)):
    name_list[i] = re.sub('[\n]','',name_list[i])
print(name_list[:3])


['于德伟', '王柏川', '袁大头']


In [6]:
word_tokenized = data['Tokenization']
print('外卖小哥' in name_list)

True


In [10]:
word = '方一凡'
word_length = len(word)
embedded_word = 0
for char in word:
    embedded_word+=model.get_vector(char)
embedded_word = embedded_word/word_length
print(embedded_word.shape)

(100,)


In [11]:
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet_upper = []
for i in alphabet:
    alphabet_upper.append(i.upper())
alphabet=[*alphabet,*alphabet_upper]

In [12]:
def embedding(data):
    word_tokenized = data['Tokenization']
    output = []
    droplist = []
    max_count = 0
    for i in range(len(word_tokenized)):
        sentence = word_tokenized[i]
        count = 0
        sentence = sentence.split(",")
        embedded_sentence = []
        flag = False
        for word in sentence:
            if flag:
                continue
            word = re.sub('[\s，。\[\]\']','',word)
            for char in word:
                if char in alphabet:
                    flag = True
                    #data.drop(i)
            if flag:
                continue
                
            if word in name_list:
                word_length = len(word)
                embedded_word = 0
                for char in word:
                    embedded_word+=model.get_vector(char)
                embedded_word = embedded_word/word_length
            else:
                try:
                    embedded_word = model.get_vector(word)
                except KeyError:
                    for char in word:
                        try:
                            embedded_char = model.get_vector(char)
                        except KeyError:
                            embedded_char = np.zeros(100)
                        embedded_sentence.append(embedded_char)
                        count += 1
                    continue
            embedded_sentence.append(embedded_word)
            count += 1
        if max_count < count: 
            max_count = count
            print(max_count)
            print(sentence)
        if flag:
            droplist.append(i)
            embedded_sentence = []
            print(data.iloc[i])
        output.append(embedded_sentence)
    data['Tensor']=output
    data.drop(index=droplist,inplace=True)
    return data

In [13]:
data = embedding(data)


2
["['真'", " '巧']"]
3
["['车'", " '没事'", " '了']"]
9
["['是'", " '你'", " '呀 '", " '没事'", " '了'", " '没'", " '事 '", " '谢谢'", " '你']"]
12
["['是'", " '不'", " '是'", " '每次'", " '开'", " '家长会'", " '来'", " '的'", " '都'", " '是'", " '他'", " '爸爸']"]
13
["['方一凡'", " '我'", " '真'", " '的'", " '不'", " '明白'", " '你'", " '脑子'", " '里面'", " '天天'", " '在'", " '想'", " '什么']"]
14
["['学习'", " '不灵'", " '打架门清'", " '一个'", " '暑假'", " '我'", " '给'", " '你'", " '报'", " '个'", " '补习班']"]
Unnamed: 0                               187
Dialogue_ID                             1018
Utterance                       alexander懂不懂
Emotion                                    3
Sentiment                                  0
Tokenization    ['alexander', '懂', '不', '懂']
Length                                     4
Name: 187, dtype: object
Unnamed: 0            189
Dialogue_ID          1019
Utterance           daddy
Emotion                 7
Sentiment               2
Tokenization    ['daddy']
Length                  1
Name: 189, dtype: object
1

In [14]:
data.shape

(80362, 8)

In [15]:
print(data['Tensor'][0])

[array([ 0.336415, -0.326726,  0.144301, -0.498075, -0.153014,  0.193251,
       -0.544008, -0.050939,  0.382979,  0.312748, -0.250881,  0.317897,
        0.358983,  0.141478,  0.045409, -0.329676,  0.142171,  0.041941,
        0.179792,  0.251903,  0.307277, -0.293694, -0.250641, -0.103327,
        0.179882,  0.181144,  0.292108,  0.222832, -0.311424,  0.341758,
        0.128687, -0.103075,  0.075947, -0.213035, -0.206132,  0.169323,
        0.005921,  0.10854 ,  0.166727, -0.129748, -0.038731, -0.150888,
       -0.134123, -0.114071,  0.291834,  0.342716, -0.015715,  0.045701,
        0.463926,  0.140579, -0.155958, -0.436883,  0.180211, -0.026786,
       -0.416272, -0.031739,  0.250457,  0.278441,  0.08179 , -0.370964,
        0.077097,  0.143798,  0.231639,  0.020945,  0.059178, -0.187715,
        0.196259, -0.259569,  0.190259,  0.040792,  0.450928,  0.328539,
       -0.045958,  0.165684, -0.119398,  0.700457, -0.175957, -0.343182,
       -0.288043,  0.049879,  0.299741,  0.448998,

In [16]:
data.to_json('train_tensor.json')

In [17]:
test_data = pd.read_csv('test_tokenized.csv')
valid_data = pd.read_csv('valid_tokenized.csv')

In [18]:
test_data = embedding(test_data)
valid_data = embedding(valid_data)

4
["['好'", " '了'", " '没事'", " '了']"]
5
["['借'", " '你'", " '这儿'", " '躲'", " '一下']"]
7
["['你'", " '踩'", " '着'", " '我'", " '的'", " '脚'", " '了']"]
9
["['我'", " '要'", " '找'", " '一个'", " '地方'", " '名字'", " '叫做'", " '爱情'", " '公寓']"]
11
["['还'", " '用'", " '找 '", " '直接'", " '坐'", " '计程车'", " '去'", " '不'", " '就'", " '好'", " '了']"]
12
["['可'", " '听说'", " '她'", " '要'", " '做'", " '一'", " '场'", " '什么'", " '婚礼'", " '的'", " '总'", " '导演']"]
15
["['每'", " '一'", " '颗'", " '圣神'", " '功丸'", " '都'", " '富'", " '含'", " '人体'", " '所'", " '需'", " '的'", " '营养'", " '元素']"]
Unnamed: 0                                                    211
Dialogue_ID                                                 27024
Utterance                                           ok 不过这次我们要改一改
Emotion                                                         3
Sentiment                                                       2
Tokenization    ['ok ', '不过', '这', '次', '我们', '要', '改', '一', '改']
Length                                                 

In [19]:
test_data.to_json('test_tensor.json')
valid_data.to_json('valid_tensor.json')

In [20]:
print(test_data['Tensor'][0][0])

[ 0.064666 -0.305517  0.113928  0.118207 -0.42618   0.150222 -0.46544
 -0.369795  0.620093  0.212185 -0.122464  0.026935  0.154822  0.238278
 -0.088391 -0.275955 -0.129593 -0.082176  0.018596  0.259984  0.00399
 -0.169681 -0.308286 -0.454857  0.343178  0.275542  0.000942  0.430707
 -0.027772 -0.314061  0.474609 -0.212621  0.211662  0.085973 -0.421056
  0.051175 -0.192907  0.07951  -0.061375 -0.233627 -0.050004  0.185441
 -0.05142  -0.754581  0.385472  0.257147 -0.144132  0.278594  0.427103
  0.150852 -0.175323 -0.456029  0.637725 -0.18438  -0.567728  0.100912
 -0.063128  0.402792  0.098915 -0.183427 -0.338206 -0.063978  0.70441
  0.189508 -0.016139 -0.045701  0.443398 -0.113004 -0.095658  0.223644
  0.299123  0.624936  0.107    -0.127483  0.262529  0.804998 -0.023288
  0.030947 -0.343273  0.128419  0.020182  0.164011  0.476149 -0.390517
 -0.283432 -0.193175 -0.04939   0.113298  0.132279 -0.225281 -0.118383
 -0.127692  0.21485  -0.01406  -0.462516 -0.329948 -0.120673  0.062445
  0.13519

In [69]:
data

,Unnamed: 0,Dialogue_ID,Utterance,Emotion,Sentiment,Tokenization,Length,Tensor
0,0,1000,真巧,0,1,"['真', '巧']",2,"[[0.312568, -0.050275, -0.24446, -0.316685, -0..."
1,1,1000,车没事了,0,1,"['车', '没事', '了']",3,"[[-0.125796, 0.295902, 0.160721, -0.171483, -0..."
2,2,1000,是你呀 没事了没事 谢谢你,0,1,"['是', '你', '呀 ', '没事', '了', '没', '事 ', '谢谢', '你']",9,"[[0.007674, 0.291153, -0.250249, -0.000174, -0..."
3,3,1000,没事没事 你也去春风,0,1,"['没', '事', '没', '事 ', '你', '也', '去', '春风']",8,"[[-0.17191, 0.295319, -0.128817, -0.16417, -0...."
4,4,1001,对,0,1,['对'],1,"[[-0.068124, 0.134782, 0.017917, 0.002805, -0...."
...,...,...,...,...,...,...,...,...
80911,80911,26265,不过你要是真有勇气离开北京,0,1,"['不过', '你', '要是', '真', '有', '勇气', '离开', '北京']",8,"[[0.256434, -0.04957, -0.077299, -0.152994, -0..."
80912,80912,26265,我反倒对你刮目相看,0,1,"['我', '反倒', '对', '你', '刮目相看']",5,"[[0.138588, 0.00559, 0.01021, 0.069461, -0.423..."
80913,80913,26265,可你走不了,0,1,"['可', '你', '走', '不', '了']",5,"[[-0.236163, 0.174653, 0.016696, -0.03327, -0...."
80914,80914,26265,我真得走了,0,1,"['我', '真', '得', '走', '了']",5,"[[0.138588, 0.00559, 0.01021, 0.069461, -0.423..."
